In [1]:
"""
Notebook for training the embedding model for the Rossler system.
Since this is not a built in example, we will need to implement our our config,
model and data handler.
=====
Distributed by: Notre Dame SCAI Lab (MIT Liscense)
- Associated publication:
url: https://arxiv.org/abs/2010.03957
doi: 
github: https://github.com/zabaras/transformer-physx
=====
"""
!nvidia-smi

zsh:1: command not found: nvidia-smi


# Environment Setup

In [2]:
%ls

data/                            train_rossler_enn.py
outputs/                         train_rossler_transformer.ipynb
train_rossler_enn.ipynb          train_rossler_transformer.py


Now lets download the training and validation data for the lorenz system. Info on wget from [Google drive](https://stackoverflow.com/questions/37453841/download-a-file-from-google-drive-using-wget). This will eventually be update to zenodo repo.

In [3]:
!mkdir data

mkdir: data: File exists


# Transformer-PhysX Rossler System


In [4]:
import sys
import os
import logging
import h5py
import paddle
import paddle.nn as nn
import numpy as np

from typing import Dict, List, Tuple

# Torch imports
from paddle.io import DataLoader, Dataset
from paddle.optimizer.lr import ExponentialDecay

# Trphysx imports
from trphysx.config.configuration_auto import AutoPhysConfig
from trphysx.embedding.training import (
    EmbeddingParser,EmbeddingTrainer
)
from trphysx.embedding.embedding_auto import AutoEmbeddingModel
from trphysx.embedding.training import AutoDataHandler

logger = logging.getLogger(__name__)

argv = []
argv = argv + ["--exp_name", "rossler"]
argv = argv + ["--training_h5_file", "./data/rossler_training.hdf5"]
argv = argv + ["--eval_h5_file", "./data/rossler_valid.hdf5"]
argv = argv + ["--stride", "16"]
argv = argv + ["--batch_size", "256"]
argv = argv + ["--block_size", "16"]
argv = argv + ["--n_train", "1024"]
argv = argv + ["--n_eval", "32"]
argv = argv + ["--epochs", "100"]

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

args = EmbeddingParser().parse(argv)
args.device = paddle.set_device("gpu:0" if paddle.is_compiled_with_cuda() else "cpu")
logger.info("Torch device: {}".format(args.device))


## Initializing Datasets and Models

Now we can use the auto classes to initialized the predefined configs, dataloaders and models. This may take a bit!

In [5]:
config = AutoPhysConfig.load_config(args.exp_name)

dataloader = AutoDataHandler.load_data_handler(args.exp_name)

# Set up data-loaders
training_loader = dataloader.createTrainingLoader(
    args.training_h5_file,
    block_size=args.block_size,
    stride=args.stride,
    ndata=args.n_train,
    batch_size=args.batch_size,
)

testing_loader = dataloader.createTestingLoader(
    args.eval_h5_file, block_size=32, ndata=args.n_eval, batch_size=8
)

# Load configuration file then init model
model = AutoEmbeddingModel.init_trainer(args.exp_name, config).to(args.device)

if args.epoch_start > 1:
    model.load_model(args.ckpt_dir, args.epoch_start)


/Users/puqing/opt/miniconda3/lib/python3.9/site-packages/paddle/fluid/dygraph/math_op_patch.py:275: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float64, but right dtype is paddle.float32, the right dtype will convert to paddle.float64
  warnings.warn(


Initialize optimizer and scheduler. Feel free to change if you want to experiment.

In [6]:
scheduler = ExponentialDecay(
    learning_rate=args.lr * 0.995 ** (args.epoch_start), gamma=0.995
)

optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(),
    learning_rate=scheduler,
    weight_decay=1e-8,
    grad_clip=paddle.nn.ClipGradByGlobalNorm(0.1),
)


## Training the Embedding Model

Train the model. No visualization here, just boring numbers. This notebook only trains for 100 epochs for brevity, feel free to train longer. The test loss here is only the recovery loss MSE(x - decode(encode(x))) and does not reflect the quality of the Koopman dynamics.

In [7]:
trainer = EmbeddingTrainer(model, args, (optimizer, scheduler))
trainer.train(training_loader, testing_loader)

/Users/puqing/opt/miniconda3/lib/python3.9/site-packages/paddle/fluid/dygraph/math_op_patch.py:275: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float64, but right dtype is paddle.float32, the right dtype will convert to paddle.float64
  warnings.warn(


Check your Google drive for checkpoints.